In [1]:
import numpy as np
import pandas as pd
import os
import librosa
import joblib
import tensorflow

In [2]:

def load_and_preprocess_data(dataset_path):
    audio_features = []

    # Check if the dataset path exists
    if not os.path.exists(dataset_path):
        print(f"Dataset path {dataset_path} does not exist.")
        return np.array([])

    # Walk through the dataset directory
    for root, dirs, files in os.walk(dataset_path):
        print(f"Processing directory: {root}")
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                try:
                    audio, sr = librosa.load(file_path, sr=16000)
                    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
                    mfcc_mean = np.mean(mfcc.T, axis=0)
                    audio_features.append(mfcc_mean)
                except Exception as e:
                    print(f'Error processing file {file_path}: {e}')

    return np.array(audio_features)

dataset_path = 'C:/audiotamperingaac/release_in_the_wild'
X = load_and_preprocess_data(dataset_path)

print(f'Extracted features shape: {X.shape}')


Processing directory: C:/audiotamperingaac/release_in_the_wild
Extracted features shape: (31779, 13)


In [28]:
df=pd.read_csv("C:/audiotamperingaac/release_in_the_wild\meta.csv")
df['label'] = df['label'].map({'spoof': 0, 'bona-fide': 1})

Y = df['label'].values
Y = Y.astype(np.float32)
print(Y)
print(f'Labels shape: {Y.shape}')


[0. 0. 0. ... 1. 1. 0.]
Labels shape: (31779,)


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {Y_train.shape}")
print(f"y_test shape: {Y_test.shape}")


X_train shape: (25423, 13)
X_test shape: (6356, 13)
y_train shape: (25423,)
y_test shape: (6356,)


In [30]:
from sklearn.preprocessing import StandardScaler
#Data normalisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import LeakyReLU, GaussianNoise
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def build_highly_optimized_model(input_shape):
    model = Sequential()

    # Input layer with Gaussian noise for regularization
    model.add(GaussianNoise(0.1, input_shape=(input_shape,)))

    # First hidden layer
    model.add(Dense(128, kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.3))

    # Second hidden layer
    model.add(Dense(64, kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.4))

    # Third hidden layer
    model.add(Dense(32, kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Optimizer with learning rate scheduling
    optimizer = Adam(learning_rate=0.001, decay=1e-6)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [32]:
print(Y_train.dtype)
print(Y_test.dtype)


float32
float32


In [33]:
input_shape = X_train.shape[1] 
model = build_highly_optimized_model(input_shape)

C:\Users\Pavan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\regularization\gaussian_noise.py:29: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\Pavan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
C:\Users\Pavan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [34]:
history = model.fit(X_train, Y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/100
636/636 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.5485 - loss: 0.9180 - val_accuracy: 0.6370 - val_loss: 0.7818
Epoch 2/100
636/636 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5997 - loss: 0.7925 - val_accuracy: 0.6370 - val_loss: 0.7414
Epoch 3/100
636/636 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6226 - loss: 0.7416 - val_accuracy: 0.6370 - val_loss: 0.7083
Epoch 4/100
636/636 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6272 - loss: 0.7087 - val_accuracy: 0.6370 - val_loss: 0.6863
Epoch 5/100
636/636 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6282 - loss: 0.6865 - val_accuracy: 0.6370 - val_loss: 0.6710
Epoch 6/100
636/636 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6269 - loss: 0.6743 - val_accuracy: 0.6370 - val_loss: 0.6643
Epoch 7/100
636/636 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6229 - loss: 0.6702 - val_accuracy: 0.6370 - val_loss: 0.6602
Epoch 8/100
636/636 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6258 - loss: 0.6658 - val_accu

In [35]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6184 - loss: 0.6657
Test Loss: 0.6641
Test Accuracy: 0.6215
